In [1]:
%load_ext autoreload
%autoreload 2

import sys
HOME = '/srv/home/christinedk/wp_internship/'
DATA_DIR = HOME + 'data/'
sys.path.append(HOME + 'collaboration/')
from config import TEMPLATES

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
from time import time
from math import log2
import json

import dateutil 
from datetime import datetime
from dateutil.relativedelta import relativedelta

from pyspark.sql import functions as f
from pyspark.sql.functions import udf, col
from pyspark.sql.types import ArrayType, StringType, IntegerType, DoubleType, FloatType

In [3]:
def talk_article_ratio(page_namespace_list):
    page_namespace_list = np.array(page_namespace_list)
    talk = np.sum(page_namespace_list==1) + 1
    article = np.sum(page_namespace_list==0) + 1
    return float(article/talk)
udf_page_talk_ratio = udf(talk_article_ratio,FloatType())

def entropy(p):
    return -sum([p[i] * log2(p[i]) for i in range(len(p))])

def contribution_fracs(page_ids):
    counts = np.unique(page_ids,return_counts=True)[1]
    cf = counts/sum(counts)
    entropy_cf = float(entropy(cf))
    return entropy_cf
udf_contribution_frac = udf(contribution_fracs,FloatType())

def read_revisions(filename, rename=False):
    revisions = pd.read_json(filename,lines=True)
    if rename:
        revisions = revisions.rename(columns = {'revision_timestamp':'event_timestamp','user_id':'event_user_id'})
    revisions['event_timestamp'] = pd.to_datetime(revisions['event_timestamp'])
    revisions = revisions.sort_values(by='event_timestamp', ascending=True)
    return revisions

def np_encoder(object):
    if isinstance(object, np.generic):
        return object.item()

In [4]:
def get_editor_features(tag_user_histories):
    
    tag_features = tag_user_histories.groupby('event_user_id')\
         .agg(f.last('num_groups').alias('num_groups'),
           f.countDistinct('page_id').alias('num_articles'),
           f.count('revision_id').alias('num_edits'),
           f.last('num_blocks_historical').alias('num_past_blocks'),
           f.last('num_curr_blocks').alias('num_curr_blocks'),
           f.sum(col("is_revert_bool")).alias('num_reverts_by_others'),
           f.sum(col('is_reverted_bool')).alias('num_reverts_of_others'),
           f.last('days_since_registration').alias('time_since_registration'),
           udf_page_talk_ratio(f.collect_list('page_namespace')).alias('talk_article_ratio'),
           udf_contribution_frac(f.collect_list('page_id')).alias('contribution_frac_entropy') 
          )
    
    return tag_features

def get_user_interactions(user_histories_1year):
    user_page_revisions = user_histories_1year.select(col('page_id'),col('event_user_id'),
                                                      col('revision_id'),col('page_namespace'))\
                                .groupBy("page_id","event_user_id").agg(
                                f.count("revision_id").alias("revisions_count"),
                                f.first("page_namespace").alias('page_namespace'))
                                #f.first("concentration_ratio").alias('concentration_ratio'))

    self_join_df = user_page_revisions.toDF(*[c + '_r' for c in user_page_revisions.columns])
    editor_interactions = user_page_revisions.join(self_join_df,[user_page_revisions.page_id == self_join_df.page_id_r,
                           user_page_revisions.event_user_id != self_join_df.event_user_id_r]).drop('page_id_r')
    
    return editor_interactions

def calculate_concentration_ratios(user_histories):
    concentration_ratio = user_histories.groupby('page_id').agg(
                                    f.countDistinct('event_user_id').alias('num_editors'),
                                    f.count('event_user_id').alias('num_revisions'))\
                                    .withColumn('concentration_ratio',col('num_editors')/col('num_revisions'))

    user_histories = user_histories.join(concentration_ratio,on='page_id')
    return user_histories

def get_directed_features(paired_interactions):
    user_article_edits = paired_interactions.groupby('event_user_id')\
                             .agg(f.sum('num_common_articles').alias('editor_pages_total'),
                             f.sum('num_revisions_articles').alias('editor_revisions_total'))

    directed = paired_interactions\
        .join(user_article_edits.select('event_user_id','editor_pages_total','editor_revisions_total'), on="event_user_id")\
        .withColumn("coedit_ratio", (col("num_common_articles") / col("editor_pages_total")))\
        .withColumn('coedit_revisions_ratio', (col('num_revisions_articles') / col('editor_revisions_total')))\
        .select('event_user_id','event_user_id_r','coedit_ratio', 'coedit_revisions_ratio')  
    
    return directed


def get_undirected_features(paired_interactions, paired_interactions_articles):
    features_all = paired_interactions.withColumn('pair',f.array_sort(f.array(col('event_user_id'),col('event_user_id_r'))))\
                            .drop_duplicates(subset=['pair'])\
                            .select('pair','num_common_pages')
    features_articles = paired_interactions_articles.withColumn('pair',f.array_sort(f.array(col('event_user_id'),col('event_user_id_r'))))\
                                    .drop_duplicates(subset=['pair'])\
                                    .select('pair','num_common_articles')#,'mean_concentration_ratio')

    undirected = features_all.join(features_articles,on='pair')
    return undirected

def calculate_collaboration_features(editor_interactions):
    #editor_interactions = calculate_concentration_ratios(editor_interactions)
    
    paired_interactions_articles = editor_interactions.filter(col('page_namespace')==0)\
                            .groupby('event_user_id','event_user_id_r')\
                            .agg(f.count("page_id").alias('num_common_articles'),
                                 f.sum('revisions_count').alias('num_revisions_articles'))\
                            .cache()
                            #f.mean('concentration_ratio').alias('mean_concentration_ratio')
                            #.filter(col('num_common_articles')>=5)
    
    paired_interactions_all = editor_interactions.groupby('event_user_id','event_user_id_r')\
                                             .agg(f.count("page_id").alias('num_common_pages'))
    
    directed_features = get_directed_features(paired_interactions_articles)
    undirected_features = get_undirected_features(paired_interactions_all, paired_interactions_articles)
                                                  
    return directed_features,undirected_features

In [5]:
user_histories = spark.read.parquet('editors/user_histories.parquet')
user_histories = user_histories.withColumn('event_timestamp',
                                           f.to_timestamp(col('event_timestamp')))\
                                .withColumn('event_user_registration_timestamp',
                                           f.to_timestamp(col('event_user_registration_timestamp')))\
                                .withColumn('is_revert_bool',col("revision_is_identity_revert").cast("long"))\
                                .withColumn('is_reverted_bool',col("revision_is_identity_reverted").cast("long"))

user_histories = user_histories.orderBy(col('event_timestamp'))

In [6]:
ls /srv/home/christinedk/wp_internship/data/

article_talk_mappings/  data_export_1402/  namespaces.csv    talk_history/
blp/                    editor_history/    negative_labels/  talk_split/
blp_articles.csv        editor_split/      page_history/
categories.csv          labels/            tag_events/


In [ ]:
for template in ['weasel','fanpov', 'autobiography','peacock','advert']:
    print(template)
    t1=time()
    labels = pd.read_csv(DATA_DIR + 'negative_labels/{}.csv'.format(template))
    labels['event_timestamp'] = pd.to_datetime(labels['event_timestamp'])

    revisions = read_revisions(DATA_DIR + 'page_history/page_history-{}-meta-info.json'.format(template))

    pages = revisions.groupby('page_id')

    print('total: ',len(labels))
    i=0
    for tag_date, page_id in labels.values:
        print('processing :',i)
        i+=1

        page_revisions = pages.get_group(page_id)
        
        # I think this can be done in a better way
        tag_users = page_revisions[page_revisions.event_timestamp <= tag_date]\
                                    .event_user_id.dropna().unique().tolist()
        tag_user_histories = user_histories.filter(col('event_timestamp')<=tag_date)\
                                           .filter(col('event_user_id').isin(tag_users))\
                                           .withColumn('days_since_registration',f.datediff(f.lit(tag_date),col('event_user_registration_timestamp')))
        #tag_user_histores = calculate_concentration_ratios(tag_user_histories)\


        editor_features = get_editor_features(tag_user_histories).toPandas().to_dict('records')

        start_date = tag_date - relativedelta(years=1)
        user_histories_1year = tag_user_histories.filter(col("event_timestamp").between(start_date,tag_date))
        editor_interactions = get_user_interactions(user_histories_1year)    

        directed_features, undirected_features = calculate_collaboration_features(editor_interactions)
        collaboration = {'directed':directed_features.toPandas().to_dict('records'), 
                         'undirected':undirected_features.toPandas().to_dict('records')}

        features= {'date':str(tag_date),'page_id':page_id,'editor':editor_features,
                         'collaboration':collaboration}
        
        with open(HOME +'negative_features/editors'+template+'.json','a+') as file:
            json.dump(features,file,default=np_encoder)
    
    print('time: ',int(time()-t1))

weasel
total:  6610
processing : 0
processing : 1
processing : 2
processing : 3
processing : 4
processing : 5
processing : 6
processing : 7
processing : 8
processing : 9
processing : 10
processing : 11
processing : 12
processing : 13
processing : 14
processing : 15
processing : 16
processing : 17
processing : 18
processing : 19
processing : 20
processing : 21
processing : 22
processing : 23
processing : 24
processing : 25
processing : 26
processing : 27
processing : 28
processing : 29
processing : 30
processing : 31
processing : 32
processing : 33
processing : 34
processing : 35
processing : 36
processing : 37
processing : 38
processing : 39
processing : 40
processing : 41
processing : 42
processing : 43
processing : 44
processing : 45
processing : 46
processing : 47
processing : 48
processing : 49
processing : 50
processing : 51
processing : 52
processing : 53
processing : 54
processing : 55
processing : 56
processing : 57
processing : 58
processing : 59
processing : 60
processing : 6

In [10]:
ls -lth /srv/home/christinedk/wp_internship/negative_features/

total 899M
-rw-r--r-- 1 christinedk wikidev 236M Mar 29 23:46 activity_peacock.json
-rw-r--r-- 1 christinedk wikidev 307M Mar 29 23:38 activity_advert.json
-rw-r--r-- 1 christinedk wikidev 114M Mar 29 23:25 activity_autobiography.json
-rw-r--r-- 1 christinedk wikidev 172M Mar 29 23:18 activity_weasel.json
-rw-r--r-- 1 christinedk wikidev  48M Mar 29 23:15 activity_fanpov.json
-rw-r--r-- 1 christinedk wikidev  11M Mar 29 21:39 talk_autobiography.json
-rw-r--r-- 1 christinedk wikidev  11M Mar 29 21:34 talk_weasel.json
-rw-r--r-- 1 christinedk wikidev 2.6M Mar 29 21:29 talk_fanpov.json


In [86]:
with open(HOME +'features/editors'+template+'.json','w') as file:
    json.dump(features,file,default=np_encoder)

In [93]:
len(features)

1160

In [8]:
template='weasel'

In [19]:
with open(HOME +'features/editors'+template+'.json','rb') as file:
    features = json.load(file)

In [20]:
print(template)
t1=time()
labels = pd.read_csv(DATA_DIR + 'labels/{}.csv'.format(template))
labels['event_timestamp'] = pd.to_datetime(labels['event_timestamp'])

revisions = read_revisions(DATA_DIR + 'page_history/page_history-{}-meta-info.json'.format(template))
pages = revisions.groupby('page_id')

weasel


In [39]:
i=len(features)

for tag_date, page_id in labels.values[1160:]:
    print('processing :',i)
    i+=1

    page_revisions = pages.get_group(page_id)

    # I think this can be done in a better way
    tag_users = page_revisions[page_revisions.event_timestamp <= tag_date]\
                                .event_user_id.dropna().unique().tolist() 
    tag_user_histories = user_histories.filter(col('event_timestamp')<=tag_date)\
                                       .filter(col('event_user_id').isin(tag_users))\
                                       .withColumn('days_since_registration',f.datediff(f.lit(tag_date),col('event_user_registration_timestamp')))
    #tag_user_histories = calculate_concentration_ratios(tag_user_histories)\

    editor_features = get_editor_features(tag_user_histories).toPandas().to_dict('records')

    start_date = tag_date - relativedelta(years=1)
    user_histories_1year = tag_user_histories.filter(col("event_timestamp").between(start_date,tag_date))
    editor_interactions = get_user_interactions(user_histories_1year)    

    directed_features, undirected_features = calculate_collaboration_features(editor_interactions)
    collaboration = {'directed':directed_features.toPandas().to_dict('records'), 
                     'undirected':undirected_features.toPandas().to_dict('records')}

    features.append({'date':str(tag_date),'page_id':page_id,'editor':editor_features,
                     'collaboration':collaboration})
        
with open(HOME +'features/editors'+template+'_v2.json','w') as file:
    json.dump(features,file,default=np_encoder)

processing : 1161
processing : 1162
processing : 1163
processing : 1164
processing : 1165
processing : 1166
processing : 1167
processing : 1168
processing : 1169
processing : 1170
processing : 1171
processing : 1172
processing : 1173
processing : 1174
processing : 1175
processing : 1176
processing : 1177
processing : 1178
processing : 1179
processing : 1180
processing : 1181
processing : 1182
processing : 1183
processing : 1184
processing : 1185
processing : 1186
processing : 1187
processing : 1188
processing : 1189
processing : 1190
processing : 1191
processing : 1192
processing : 1193
processing : 1194
processing : 1195
processing : 1196
processing : 1197
processing : 1198
processing : 1199
processing : 1200
processing : 1201
processing : 1202
processing : 1203
processing : 1204
processing : 1205
processing : 1206
processing : 1207
processing : 1208
processing : 1209
processing : 1210
processing : 1211
processing : 1212
processing : 1213
processing : 1214
processing : 1215
processing

In [ ]:
# 231 250

In [ ]:
251 266

In [8]:
template = 'fanpov'

In [36]:
%%time

features = []
pages = revisions.groupby('page_id')

print('total: ',len(labels))
i=0

for tag_date, page_id in labels.values[:10]:
    print('\t processing :',i)
    i+=1
    
    page_revisions = pages.get_group(page_id)    
    tag_users = page_revisions[page_revisions.event_timestamp <= tag_date]\
                                .event_user_id.dropna().unique().tolist() # I think this can be done in a better way
    tag_user_histories = user_histories.filter(col('event_timestamp')<tag_date)\
                                       .filter(col('event_user_id').isin(tag_users))\
                                       .withColumn('days_since_registration',f.datediff(f.lit(tag_date),col('event_user_registration_timestamp')))

    editor_features = get_editor_features(tag_user_histories).toPandas().to_dict('records')

    start_date = tag_date - relativedelta(years=1)
    user_histories_1year = tag_user_histories.filter(col("event_timestamp").between(start_date,tag_date))
    editor_interactions = get_user_interactions(user_histories_1year)    

    directed_features, undirected_features = calculate_collaboration_features(editor_interactions)
    collaboration = {'directed':directed_features.toPandas().to_dict('records'), 
                     'undirected':undirected_features.toPandas().to_dict('records')}
    
    print('\t\t num records: ',len(collaboration['undirected']))

    features.append({'date':tag_date,'page_id':page_id,'editor':editor_features,
                     'collaboration':collaboration})

total:  721
processing : 0
num records:  5051
processing : 1
num records:  5702
processing : 2
num records:  506
processing : 3
num records:  1615
processing : 4
num records:  1000
processing : 5
num records:  7677
processing : 6
num records:  7959
processing : 7
num records:  2010
processing : 8
num records:  1218
processing : 9
num records:  32
CPU times: user 4.35 s, sys: 1.08 s, total: 5.43 s
Wall time: 28min 17s
